In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
from typing import Dict


def clean_json(json_str: str) -> Dict:
    aska = json_str[::-1]
    edge_inda = - aska.index("}")
    if edge_inda != 0:
        json_str = json_str[:edge_inda] + ']'
    else:
        json_str += ']'
    return json.loads(json_str)


def get_characts(obj):
    characts = set()
    if not pd.isna(obj['Характеристики СТЕ']):
        for charact in clean_json(obj['Характеристики СТЕ']):
            try:
                characts.add((charact['Name'], charact['Value']))
            except KeyError:
                pass
    return characts


data = pd.read_excel('/content/drive/MyDrive/tenderhack/dataset_base.xlsx')  # FILEPATH
id2obj = {}
categories = {}
connected = {}
for iterr, st in data.iterrows():
    id2obj[st['Идентификатор СТЕ']] = st

    if st['Категория'] not in categories:
        categories[st['Категория']] = []
    categories[st['Категория']].append(st['Идентификатор СТЕ'])

    if st['Категория'] not in connected:
        connected[st['Категория']] = {}
    if not pd.isna(st['Другая продукция в контрактах']) and len(st['Другая продукция в контрактах'].strip()) > 0:
        st_others = st['Другая продукция в контрактах']
        st_others = clean_json(st_others)
        for prod in st_others:
            try:
                connected_cat = id2obj[prod['OtherSkuId']]['Категория']
                if connected_cat not in connected[st['Категория']]:
                    connected[st['Категория']][connected_cat] = 0
                connected[st['Категория']][connected_cat] += 1
            except KeyError:
                pass

In [ ]:
data.head()

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
0,20528973,"Светодиодная лента 2835, ULTRA, 12 Вольт, 60 l...",Лампы светодиодные,NaN,"[{""Name"":""Цвет свечения"",""Id"":-506425117,""Valu...","[{""Name"":""Москва""}]",NaN,"[{""SupplierId"":2310783,""Name"":""ООО \""ЛЮМЕНВАТТ...",NaN,NaN,1,12106442.0,01.10.05.02.04,2835ULTRA,NaN
1,34860341,Светодиодный светильник Geniled ЛПО 1200?180?...,Настенно-потолочные светильники,NaN,"[{""Name"":""Вид климатического исполнения"",""Id"":...","[{""Name"":""Свердловская""},{""Name"":""Москва""}]",1.0,"[{""SupplierId"":1559525,""Name"":""Общество с огра...",РОССИЯ,NaN,41,12114082.0,01.10.05.03.01.05,ЛПО 1200?180?40 40Вт 5000К Опал,NaN
2,34172198,Елка пристенно-угловая Графская премиум 210 см...,Предметы внутреннего интерьера,NaN,"[{""Name"":""Цвет"",""Id"":340085190,""Value"":""Зелены...","[{""Name"":""Москва""},{""Name"":""Все регионы""}]",NaN,"[{""SupplierId"":1348988,""Name"":""ИП Чепало Оксан...",NaN,NaN,5,68093.0,01.20.08,Елка пристенно-угловая Графская премиум 210 см.,NaN
3,1318014,Картридж HP CE278A/Canon 728 LJ P1560/1566/160...,Расходные материалы и комплектующие для лазерн...,NaN,"[{""Name"":""Тип расходника"",""Id"":317455168,""Valu...","[{""Name"":""Москва""},{""Name"":""Московская""}]",9.0,"[{""SupplierId"":1111141,""Name"":""Общество с огра...",NaN,"[{""OtherSkuId"":1398583,""OtherSkuName"":""013R006...",6,12105204.0,01.13.13.01.01,SF-CE278(2K),NaN
4,1251411,Штамп самонаборный Pr.C20/3-Set 38х14 (аналог ...,Штемпели,NaN,"[{""Name"":""Наличие рамки"",""Id"":366100137,""Value...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",71.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",АВСТРИЯ,"[{""OtherSkuId"":1171023,""OtherSkuName"":""Berling...",72,12120460.0,01.15.06.07,Штамп самонаборный пласт. 3стр. Pr.C20/3-Set 3...,"[{""Cost"":454.67000},{""Cost"":620.00000},{""Cost""..."


In [ ]:
pairs_df = pd.read_csv('/content/drive/MyDrive/tenderhack/sample_pairs.csv')

In [ ]:
import pickle
with open('/content/drive/MyDrive/tenderhack/charact_dict.pickle', 'rb') as handle:
    all_characts = pickle.load(handle)

In [ ]:
len(all_characts)

58343

In [ ]:
data_pairs = pairs_df.sample(50000)

In [ ]:
data_pairs.target.value_counts()

0    25038
1    24962
Name: target, dtype: int64

In [ ]:
one_hotted = []

for ind, st in data_pairs.iterrows():
  p1, p2, target = st['p1'], st['p2'], st['target']
  onehot_char = ['0' for i in range(len(all_characts))]
  try:
    characts1 = get_characts(id2obj[p1])
    characts2 = get_characts(id2obj[p2])
    same_characts = characts1 & characts2
    for ch_name, ch_val in same_characts:
      onehot_char[all_characts[ch_name]] = '1'
  except KeyError:
    pass
  one_hotted.append(''.join(onehot_char))


data_pairs['one_hotted'] = one_hotted

In [ ]:
ressa = 0
for i in data_pairs['one_hotted']:
  ressa |= int(i, 2)

In [ ]:
ressa = str(bin(ressa))[2:]
ressa = '0'*(len(all_characts)-len(ressa)) + ressa

In [ ]:
nice = []
for i in range(len(ressa)):
  if ressa[i]=="1":
    nice.append(i)

In [ ]:
len(nice)

In [ ]:
ids2charact = {all_characts[charact]:charact for charact in all_characts}
nice_characts = [ids2charact[i] for i in nice]

In [ ]:
one_hotted_nice = []
for i in one_hotted:
  ideal = ''
  for nice_inda in nice:
    ideal+=i[nice_inda]
  one_hotted_nice.append(list(ideal))


In [ ]:
data_pairs['one_hotted'] = one_hotted_nice

In [ ]:
data_pairs.to_csv('/content/drive/MyDrive/tenderhack/pairwise.csv')

In [ ]:
commons = []
for ind, st in data_pairs.iterrows():
  p1, p2 = st['p1'], st['p2']
  try:
    connected1 = connected[id2obj[p1]['Категория']]
    connected2 = connected[id2obj[p2]['Категория']]
    common = len(set(connected1)&set(connected2))
    commons.append(common)
  except KeyError:
    commons.append(0)
data_pairs['common'] = commons

In [ ]:
data_pairs.common.value_counts()

0      14890
1       2256
2       1519
3       1072
4        806
       ...  
302        1
365        1
491        1
523        1
297        1
Name: common, Length: 376, dtype: int64

In [ ]:
data_pairs.to_csv('/content/drive/MyDrive/tenderhack/pairwise.csv')

In [ ]:
p2s = []
for p2 in data_pairs.p2:
  costs = id2obj[p2]['Цена']
  if pd.isna(costs) or not costs:
    p2s.append(0)
  else:
    all_costs = []
    for cost in clean_json(costs):
      all_costs.append(cost['Cost'])
    p2s.append(all_costs[len(all_costs)//2])


In [ ]:
data_pairs['cost'] = p2s

In [ ]:
contracts = []
for p2 in data_pairs.p2:
  if pd.isna(id2obj[p2]['Кол-во заключенных контрактов']):
    contracts.append(0)
  else:
    contracts.append(id2obj[p2]['Кол-во заключенных контрактов'])
data_pairs['contracts'] = contracts

In [ ]:
views = []
for p2 in data_pairs.p2:
  if pd.isna(id2obj[p2]['Просмотры']):
    views.append(0)
  else:
    views.append(id2obj[p2]['Просмотры'])
data_pairs['views'] = views

In [ ]:
with open('/content/drive/MyDrive/tenderhack/median_cost.pickle', 'rb') as handle:
    mid_cost = pickle.load(handle)

In [ ]:
def get_cost(obj):
  costs = obj['Цена']
  if pd.isna(costs) or not costs:
    return 0
  else:
    all_costs = []
    for cost in clean_json(costs):
      all_costs.append(cost['Cost'])
    return all_costs[len(all_costs)//2]

In [ ]:
tilts = []
for inda, st in data_pairs.iterrows():
  try:
    p1, p2 = id2obj[st['p1']], id2obj[st['p2']]
    difa1 = get_cost(p1) - mid_cost[p1['Категория']]
    difa2 = get_cost(p2) - mid_cost[p2['Категория']]
    tilts.append(abs(difa1-difa2))
  except KeyError:
    tilts.append(0)
data_pairs['tilt'] = tilts


In [ ]:
data_pairs.to_csv('/content/drive/MyDrive/tenderhack/pairwise.csv')

In [ ]:
data_pairs = data_pairs[['target', 'common', 'contracts', 'views', 'tilt', 'one_hotted']]

In [ ]:
data_pairs.corr()

,target,common,contracts,views,tilt
target,1.000000,-0.075676,0.315530,0.142916,-0.084136
common,-0.075676,1.000000,0.001626,0.000586,-0.037099
contracts,0.315530,0.001626,1.000000,0.559462,-0.043518
views,0.142916,0.000586,0.559462,1.000000,-0.008832
tilt,-0.084136,-0.037099,-0.043518,-0.008832,1.000000


In [ ]:
data_pairs.to_csv('/content/drive/MyDrive/tenderhack/pairwise.csv')

In [ ]:
one_hotted_list = []
for i in data_pairs.one_hotted:
    one_hotted_list.append(list(map(int, list(i))))

In [ ]:
one_hotted_df = pd.DataFrame(one_hotted_list, columns = nice_characts)

In [ ]:
base_df = data_pairs[['target', 'common', 'contracts', 'views', 'tilt']]

In [ ]:
base_df.target[:10]

161456    1
167799    1
442312    0
453719    0
176930    1
268404    0
178765    1
411509    0
268478    0
61129     1
Name: target, dtype: int64

In [ ]:
total_data = base_df
for column in one_hotted_df.columns:
  total_data[column] = list(one_hotted_df[column])

In [ ]:
total_data.head()


,target,common,contracts,views,tilt,Напряжение,Вид товаров,Вид энергетического оборудования,Цвет,Материал,Вид продукции,Высота,Гарантийный срок,Тип расходника,Производитель расходных материалов для печатающей техники,Ресурс,Тип,Вид обложки,Фиксирующая резинка,Стиль обложки,Размер,Количество листов,Назначение,Ширина,Страна производства,Страна производитель,Схема вложения,Масса,Упаковка,Длина,Вес,Страна происхождения,Состав ткани,Подарочная упаковка,Количество штук в упаковке,Формат,Индивидуальная упаковка,Внутренний блок,Количество дизайнов в упаковке,Перфорация,...,Легко затачиваются,Длина карандаша,Тип зажима,Количество цветов в пачке,Чернила на масляной основе,Формат папки ориентирован,Вид инженерно-строительные товары,Арочный механизм папки открывается на 180 градусов,Концентрат,Вид письменных принадлежностей,"Вид система охраны, наблюдения, обеспечения безопасности",Вид расходных материалов,Для левшей,В разобранном виде,Опудренные,Вид лакокрасочного материала,Год издания,Тип установки,Производительность охлаждения,Форма пучка кисти,Количество кистей,"Вид информационно-технологический товар, средство связи, оргтехника, электроника (включая программное обеспечение)",Совместимые модели принтеров,Наличие в салоне,Цветность света,Произвольная регулировка направления подсветки,Отсутствие мерцания и шумов,Источник питания,Материал жил,Материал вставки,Регулировка по высоте,Защитные свойства,Температура охлаждения воды,Угол,"Вид инвентарь, инструменты",Ножка-подставка,Толщина лдсп,Персонаж,Тип сценической одежды,Материал задника
161456,1,106,9.0,18,3156.56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
167799,1,75,9.0,18,2886.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
442312,0,149,3.0,5,387.90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
453719,0,126,2.0,7,55.30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
176930,1,60,9.0,18,2910.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
total_data.corr()

,target,common,contracts,views,tilt,Напряжение,Вид товаров,Вид энергетического оборудования,Цвет,Материал,Вид продукции,Высота,Гарантийный срок,Тип расходника,Производитель расходных материалов для печатающей техники,Ресурс,Тип,Вид обложки,Фиксирующая резинка,Стиль обложки,Размер,Количество листов,Назначение,Ширина,Страна производства,Страна производитель,Схема вложения,Масса,Упаковка,Длина,Вес,Страна происхождения,Состав ткани,Подарочная упаковка,Количество штук в упаковке,Формат,Индивидуальная упаковка,Внутренний блок,Количество дизайнов в упаковке,Перфорация,...,Легко затачиваются,Длина карандаша,Тип зажима,Количество цветов в пачке,Чернила на масляной основе,Формат папки ориентирован,Вид инженерно-строительные товары,Арочный механизм папки открывается на 180 градусов,Концентрат,Вид письменных принадлежностей,"Вид система охраны, наблюдения, обеспечения безопасности",Вид расходных материалов,Для левшей,В разобранном виде,Опудренные,Вид лакокрасочного материала,Год издания,Тип установки,Производительность охлаждения,Форма пучка кисти,Количество кистей,"Вид информационно-технологический товар, средство связи, оргтехника, электроника (включая программное обеспечение)",Совместимые модели принтеров,Наличие в салоне,Цветность света,Произвольная регулировка направления подсветки,Отсутствие мерцания и шумов,Источник питания,Материал жил,Материал вставки,Регулировка по высоте,Защитные свойства,Температура охлаждения воды,Угол,"Вид инвентарь, инструменты",Ножка-подставка,Толщина лдсп,Персонаж,Тип сценической одежды,Материал задника
target,1.000000,-0.075676,0.315530,0.142916,-0.084136,-0.006315,-0.054771,-0.004465,0.059239,-0.018414,-0.174507,-0.007734,-0.029968,0.013438,0.015517,0.011851,-0.006315,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.008931,-0.027537,-0.029633,-0.014122,-0.004465,-0.010938,-0.008931,-0.009985,-0.008931,-0.004465,-0.006315,-0.004465,-0.016710,-0.006315,-0.007734,-0.006315,-0.006315,...,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.009985,-0.004465,-0.004465,-0.004465,-0.004465,-0.006315,-0.004465,-0.004465,-0.004465,-0.004465,-0.007734,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465,-0.006315,-0.004465,-0.004465,-0.004465,-0.004465,-0.004465
common,-0.075676,1.000000,0.001626,0.000586,-0.037099,-0.000736,0.120587,0.002759,0.133264,0.021183,0.017827,0.016668,-0.013036,0.042830,0.040870,0.029181,0.009619,0.014047,0.019004,0.019004,-0.000444,0.019004,0.009089,0.019133,0.002175,0.017942,0.036705,-0.000368,0.051255,0.017455,-0.003688,0.000370,0.010691,0.002931,0.014809,0.043595,0.011560,0.000111,0.013879,0.018841,...,0.014581,0.014581,0.014809,0.011682,0.023962,0.018394,0.031036,0.018394,0.009547,0.018241,0.013437,0.020189,0.023962,0.008784,0.011149,0.028156,-0.005261,0.006954,0.006954,0.009089,0.009089,0.005734,0.014276,-0.002122,0.012521,0.012521,0.012521,0.012521,0.014352,0.010310,-0.002198,0.010691,0.006954,0.020224,0.021537,0.010310,-0.001741,-0.000673,-0.000673,0.010310
contracts,0.315530,0.001626,1.000000,0.559462,-0.043518,-0.002222,-0.023030,-0.000974,0.017982,-0.005755,-0.067508,-0.002721,-0.020069,0.004240,0.004896,0.003739,-0.004332,-0.003959,-0.003959,-0.003959,-0.002168,-0.003959,-0.000377,-0.006425,-0.013370,-0.011957,-0.007612,-0.002765,-0.007017,-0.003441,-0.006717,0.000439,-0.003362,-0.001800,-0.000974,0.000183,-0.001800,-0.006857,-0.005599,-0.004754,...,-0.003362,-0.003362,-0.000974,0.006188,-0.002765,0.005591,-0.004047,0.005591,-0.003362,-0.002765,-0.002765,-0.004332,-0.002765,-0.003959,-0.000377,0.000219,-0.006857,-0.002765,-0.002765,-0.000377,-0.000377,-0.002168,-0.002765,-0.003959,-0.003959,-0.003959,-0.003959,-0.003959,-0.003362,-0.003959,-0.001571,-0.003362,-0.002765,0.007979,-0.003488,-0.003959,-0.002168,-0.003959,-0.003959,-0.003959
views,0.142916,0.000586,0.559462,1.000000,-0.008832,-0.003185,-0.006266,0.000098,0.008714,-0.005956,-0

In [ ]:
total_data.to_csv('/content/drive/MyDrive/tenderhack/pairwise_full.csv')

In [ ]:
!pip install xgboost

In [ ]:
from sklearn import datasets
import xgboost as xgb


In [ ]:
X = total_data.drop(['target'], axis =1)
y = total_data.target

In [ ]:
y.head()

161456    1
167799    1
442312    0
453719    0
176930    1
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.9234

In [ ]:
clf.coef_.max()

0.36786484100326206

In [ ]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [ ]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [ ]:
model = xgb.train(param, D_train, steps)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])


print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9998010741993236
Recall = 0.9997989949748743
Accuracy = 0.9998


In [ ]:
model.save_model("/content/drive/MyDrive/tenderhack/model_xgb.json")

In [ ]:
model_xgb_2 = xgb.Booster()
model_xgb_2.load_model("model.json")